In [8]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sentinelhub import SHConfig, SentinelHubRequest, MimeType, CRS, BBox, bbox_to_dimensions, DataCollection
from sentinelhub.time_utils import parse_time
import geopandas as gpd

In [9]:
config = SHConfig()

In [10]:
config.sh_client_id = 'your-client-id'
config.sh_client_secret = 'your-client-secret'

In [11]:
# aoi_geometry = {
#     "type": "Polygon",
#     "coordinates": [
#         [
#             [88.6325, 25.6215],
#             [88.6325, 25.7362],
#             [88.8074, 25.7362],
#             [88.8074, 25.6215],
#             [88.6325, 25.6215]
#         ]
#     ]
# }

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              88.7210993420249,
              25.853760881617248
            ],
            [
              88.7210993420249,
              25.859930857145514
            ],
            [
              88.71485314753073,
              25.859930857145514
            ],
            [
              88.71485314753073,
              25.853760881617248
            ],
            [
              88.7210993420249,
              25.853760881617248
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'coordinates': [[[88.7210993420249, 25.853760881617248],
      [88.7210993420249, 25.859930857145514],
      [88.71485314753073, 25.859930857145514],
      [88.71485314753073, 25.853760881617248],
      [88.7210993420249, 25.853760881617248]]],
    'type': 'Polygon'}}]}

In [12]:
start_date = '2023-11-01'
end_date = '2024-06-30'

In [13]:
aoi_bbox = BBox(bbox=[88.7210993420249, 25.853760881617248, 88.71485314753073, 25.859930857145514], crs=CRS.WGS84)
resolution = 10  # m/pix
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

In [14]:
request = SentinelHubRequest(
    data_folder='sentinel-2-images',
    evalscript='''
        //VERSION=3
        function setup() {
            return {
                input: ["B02", "B03", "B04", "B08"],
                output: { bands: 3 }
            };
        }

        function evaluatePixel(sample) {
            return [sample.B04, sample.B03, sample.B02];
        }
    ''',
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,
            time_interval=(start_date, end_date),
            mosaicking_order='mostRecent'
        )
    ],
    responses=[
        SentinelHubRequest.output_response('default', MimeType.PNG)
    ],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config
)

In [15]:
response = request.get_data(save_data=True)
image = np.array(response[0])
plt.imshow(image)
plt.show()

InvalidClientError: (invalid_client) Invalid client or Invalid client credentials